In [85]:
import os
import time
import shutil

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import tensorflow.keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense

Constants

In [72]:
SELF_PATH = os.getcwd()
PATH_TO_FILE_STREAM = 'G:\\Shared drives\\P-Sick'
PATH_TO_IMAGES = os.path.join(PATH_TO_FILE_STREAM, 'small\\Florence\\20180917a_jpgs\\jpgs')
PATH_TO_TAG_CSV = os.path.join(SELF_PATH, '../tagging_data.csv')
TEST_TO_TRAIN_RATIO = 0.3
TRAINING_IMAGE_FOLDER = "../_training_images"
TESTING_IMAGE_FOLDER = "../_testing_images"

These are a list of the ids for impact
```
NoneId:0  
SwashId:1  
CollisionId:2  
OverwashId:3  
InundationId:4
```

# Getting the images

1. We need to load the data from the csv
2. Split the images up into training and test set, and then place them in seperate folders.

First lets load the csv that has all the completely tagged image tags.

In [73]:
df_image_tags = pd.read_csv(PATH_TO_TAG_CSV)
df_impact_images = df_image_tags[['image_id','impact']]

In [86]:
df_training_images, df_testing_images = train_test_split(df_impact_images, test_size=TEST_TO_TRAIN_RATIO, random_state=420)

Before we copy the images, remove all files within these folders

In [75]:
for file in os.listdir(TRAINING_IMAGE_FOLDER):
    os.remove(os.path.join(TRAINING_IMAGE_FOLDER,file)) 

for file in os.listdir(TESTING_IMAGE_FOLDER):
    os.remove(os.path.join(TESTING_IMAGE_FOLDER,file)) 

Copy our training and test images into their respective folders, while keeping metadata with copy2

In [76]:
for image in df_testing_images['image_id']:
    shutil.copy2(os.path.join(PATH_TO_IMAGES,image),  os.path.join(TRAINING_IMAGE_FOLDER,image))

for image in df_testing_images['image_id']:
    shutil.copy2(os.path.join(PATH_TO_IMAGES,image),  os.path.join(TESTING_IMAGE_FOLDER,image))

print("Moved training and test images to their respective folders")

Moved training and test images to their respective folders


# Preprocessing the images

1. Seperate the `image_id` and the `impact` into different np arrays for Keras

In [77]:
image_samples = df_impact_images['image_id'].to_numpy()
image_labels = df_impact_images['impact'].to_numpy()

In [ ]:
model = Sequential([
    Dense(16,input_shape = ())
])